In [ ]:
import pandas as pd
import os

def extract_certain_day_related_news(market_data_path, news_dir, day):
    df_market = pd.read_excel(market_data_path)  # 读取 Excel
    df_market['date'] = pd.to_datetime(df_market['datetime']).dt.date  # 统一日期格式

    # 读取新闻数据
    df_news = pd.concat([
        pd.read_json(os.path.join(news_dir, filename))
        .assign(instrument=filename.split('.')[0])
        for filename in os.listdir(news_dir) if filename.endswith(".json")
    ], ignore_index=True)

    # 统一新闻数据的日期格式
    df_news['date'] = pd.to_datetime(df_news['date']).dt.date

    # 找到市场数据中当天有数据的股票
    stocks_on_day = df_market.loc[df_market['date'] == pd.to_datetime(day).date(), 'instrument'].unique()

    # 筛选新闻数据，要求日期为 day，且 instrument 在 stocks_on_day 中
    df_filtered_news = df_news[(df_news['date'] == pd.to_datetime(day).date()) & (df_news['instrument'].astype(int).isin(stocks_on_day))]
        
    return df_filtered_news


market_data_path = "2019Q4股票预测.xlsx"
news_dir = "/mnt/disk2/xinghua.jia/workspace/finbot/resources/data/ablation_recommend/csi300news_2019/seq@q25_b3_sft_sum@finbert_valuesimplex_senti"
day = "2019-10-08"
df_filtered_news = extract_certain_day_related_news(market_data_path, news_dir, day)
df_filtered_news.to_json(f"{day}_相关新闻片段.json", orient="records",indent=4,index=False,force_ascii=False)